In [1]:
import pandas as pd
import numpy as np
import re as re

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',100)

In [2]:
# Take in Excel

itemized = pd.read_excel('/Users/danielsagher/Dropbox/Documents/projects/nautical_reconciliation/excel/Nautical 11-2024.xlsx', header=2, sheet_name=0)
invoice_data = pd.read_excel('/Users/danielsagher/Dropbox/Documents/projects/nautical_reconciliation/excel/Nautical 11-2024.xlsx', sheet_name=1)
qbo = pd.read_excel('/Users/danielsagher/Dropbox/Documents/projects/nautical_reconciliation/excel/QBO_customers.xlsx')
amt = pd.read_excel('/Users/danielsagher/Dropbox/Documents/projects/nautical_reconciliation/excel/Exensiv.xlsx', sheet_name='AMT')
gp_acoustics = pd.read_excel('/Users/danielsagher/Dropbox/Documents/projects/nautical_reconciliation/excel/Exensiv.xlsx', sheet_name='GPAcoustics')

# **Invoice Data** <-> **QBO**

## - Compare **Invoice Data** [`Customer PO #`] to **QBO** [`Display_Name`]

In [3]:
# Merge QuickBooks and FedEx to find shared values

#! Check if Display name is the only key to compare against

qbo_found = pd.merge(qbo, invoice_data, right_on='Customer PO #', left_on= 'Display_Name', how='inner')

In [4]:
# Number of PO's Found in QuickBooks

print(f'Unique Found POs: {len(qbo_found['Display_Name'].drop_duplicates())}',
      f'Total Found POs: {len(qbo_found)}', sep='\n')

Unique Found POs: 6
Total Found POs: 2218


### - Add not found values to new table: `not_found`

In [5]:
# Add unique Customer PO's not found in quickbooks to a unique list

lst = set()
for i in invoice_data['Customer PO #']:
    if i not in list(qbo_found['Display_Name'].unique()):
        lst.add(i)

In [6]:
len(lst)

84

In [7]:
not_found = pd.DataFrame()
not_found['Customer PO #'] = pd.DataFrame(lst)
not_found = not_found.merge(invoice_data[['Customer PO #',  
                            'Reference',
                            'Reference 2',
                            'Total Charges',
                            'Receiver Name',
                            'Receiver Company',
                            'Receiver Address',
                            'Tracking #']],
                             on='Customer PO #', how='left')
not_found

,Customer PO #,Reference,Reference 2,Total Charges,Receiver Name,Receiver Company,Receiver Address,Tracking #
0,113-9410880-0465011,14324,NaN,106.98,CHIP BARTLETT,CHIP BARTLETT,43 PASEO VESPERTINO,281378830569
1,700670006/SD266461,10558,NaN,4.92,MUSICAL FULFILLMENT OH,MUSICAL FULFILLMENT OH,1040 N WYNN RD,280881667358
2,Nautical/Arrived/Splenda,ARRCO-22724-00027,NaN,5.10,BETH RUGE,SPLENDA STEVIA,14390 CLAY TERRACE BLVD,418132348850
3,Nautical/Arrived/Splenda,ARRCO-22724-00027,NaN,4.92,JENNIFER,"CAROLINE R KLUTTZ, MS, RDN, LDN",8810 BLAKENEY PROFESSIONAL DR,418132349099
4,Nautical/Arrived/Splenda,ARRCO-22724-00027,NaN,5.23,Nautical Fulfillment,Nautical Fulfillment,16100 W 116th St,418132350276
...,...,...,...,...,...,...,...,...
2750,RECESS-22124-00019,Nautical/Recess/Rockstar,NaN,9.96,COMMUNITY MANAGER,WEWORK - 400 CONCAR DR,400 CONCAR DR,426933225007
2751,RECESS-22124-00019,Nautical/Recess/Rockstar,NaN,9.96,COMMUNITY MANAGER,WEWORK - 1 BELVEDERE,1 BELVEDERE PL,426933223493
2752,RECESS-22124-00019,Nautical/Recess/Rockstar,NaN,20.62,COMMUNITY MANAGER,WEWORK - 450 PARK AVE S,450 PARK AVE S,426933225132
2753,RECESS-22124-00019,Nautical/Recess/Rockstar,NaN,9.45,COMMUNITY MANAGER,WEWORK - 33 ARCH ST,33 ARCH ST,426933224868


# **Invoice Data** [`Reference`] <-> **Extensiv** [`Reference`]

## - For all **Invoice Data** [`Customer PO #`] not in **QBO** [`Display Name`]

### - Create RegEx token function 

In [8]:
# Create Regex Tokens

def reg_tokenizer(value):

    '''
    Input: Value of each column in Reference and Reference 2
    Output: RegEx object
    '''

    with_letters = re.sub(r'[a-zA-Z]+',r'\\w+' , str(value))

    with_numbers = re.sub(r'\d+', r'\\d+', with_letters)

    with_spaces = re.sub(r'\s+', r'\\s+', with_numbers)

    final = re.compile(with_spaces)
    
    return final

In [9]:
# RegEx Test

for i in range(len(not_found['Customer PO #'])):

    test = reg_tokenizer(not_found['Customer PO #'][i])
    
    match = re.fullmatch(test, str(not_found['Customer PO #'][i]))
    
    # print(f'{test}, original: {not_found['Customer PO #'][i]}, match: {match.group(0)}')

### - Add RegEx column to `not_found` with [Reference] patterns

In [10]:
# Add Pattern column

token_lst = []

for i in not_found['Reference']:
    token_lst.append(reg_tokenizer(i))

not_found['Pattern'] = token_lst


In [11]:
# Fedex columns with added Reference pattern

not_found

,Customer PO #,Reference,Reference 2,Total Charges,Receiver Name,Receiver Company,Receiver Address,Tracking #,Pattern
0,113-9410880-0465011,14324,NaN,106.98,CHIP BARTLETT,CHIP BARTLETT,43 PASEO VESPERTINO,281378830569,re.compile('\\d+')
1,700670006/SD266461,10558,NaN,4.92,MUSICAL FULFILLMENT OH,MUSICAL FULFILLMENT OH,1040 N WYNN RD,280881667358,re.compile('\\d+')
2,Nautical/Arrived/Splenda,ARRCO-22724-00027,NaN,5.10,BETH RUGE,SPLENDA STEVIA,14390 CLAY TERRACE BLVD,418132348850,re.compile('\\w+-\\d+-\\d+')
3,Nautical/Arrived/Splenda,ARRCO-22724-00027,NaN,4.92,JENNIFER,"CAROLINE R KLUTTZ, MS, RDN, LDN",8810 BLAKENEY PROFESSIONAL DR,418132349099,re.compile('\\w+-\\d+-\\d+')
4,Nautical/Arrived/Splenda,ARRCO-22724-00027,NaN,5.23,Nautical Fulfillment,Nautical Fulfillment,16100 W 116th St,418132350276,re.compile('\\w+-\\d+-\\d+')
...,...,...,...,...,...,...,...,...,...
2750,RECESS-22124-00019,Nautical/Recess/Rockstar,NaN,9.96,COMMUNITY MANAGER,WEWORK - 400 CONCAR DR,400 CONCAR DR,426933225007,re.compile('\\w+/\\w+/\\w+')
2751,RECESS-22124-00019,Nautical/Recess/Rockstar,NaN,9.96,COMMUNITY MANAGER,WEWORK - 1 BELVEDERE,1 BELVEDERE PL,426933223493,re.compile('\\w+/\\w+/\\w+')
2752,RECESS-22124-00019,Nautical/Recess/Rockstar,NaN,20.62,COMMUNITY MANAGER,WEWORK - 450 PARK AVE S,450 PARK AVE S,426933225132,re.compile('\\w+/\\w+/\\w+')
2753,RECESS-22124-00019,Nautical/Recess/Rockstar,NaN,9.45,COMMUNITY MANAGER,WEWORK - 33 ARCH ST,33 ARCH ST,426933224868,re.compile('\\w+/\\w+/\\w+')


### - Compare `not found`[Pattern] to values in Extensiv Tables

In [12]:
# Compare Reference Patterns against every value in Extensiv Table

def find_col_match(ext_table, ref_pattern):

    col_lst = set()

    for col in ext_table.columns:

        for value in ext_table[col]:

            if re.fullmatch(ref_pattern,str(value)):

                col_lst.add(col)
                break
            
            else:
                break

    if len(col_lst) != 0:
        return col_lst
    else:
        return None


In [13]:
# Test \d+ to AMT table 

find_col_match(amt, not_found['Pattern'][1])



{'AsnCandidate',
 'Column1',
 'CreatedByIdentifier.Id',
 'Custom.TotalResults',
 'CustomerIdentifier.Id',
 'Index',
 'LastModifiedByIdentifier.Id',
 'LoadedState',
 'OrderId',
 'ParcelLabelType',
 'RouteCandidate',
 'RoutingInfo.BillOfLading',
 'ShipTo.Zip',
 'Status',
 'TransactionEntryType',
 'WarehouseTransactionSourceType'}

In [14]:
# Check AMT table

amt[['AsnCandidate',
 'Column1',
 'CreatedByIdentifier.Id',
 'Custom.TotalResults',
 'CustomerIdentifier.Id',
 'Index',
 'LastModifiedByIdentifier.Id',
 'LoadedState',
 'OrderId',
 'ParcelLabelType',
 'RouteCandidate',
 'RoutingInfo.BillOfLading',
 'ShipTo.Zip',
 'Status',
 'TransactionEntryType',
 'WarehouseTransactionSourceType']].head()

,AsnCandidate,Column1,CreatedByIdentifier.Id,Custom.TotalResults,CustomerIdentifier.Id,Index,LastModifiedByIdentifier.Id,LoadedState,OrderId,ParcelLabelType,RouteCandidate,RoutingInfo.BillOfLading,ShipTo.Zip,Status,TransactionEntryType,WarehouseTransactionSourceType
0,0,0,10,1044,11,1,10,0,363,0,0,17,52404,1,1,1
1,0,0,10,1044,11,1,10,0,363,0,0,17,52404,1,1,1
2,0,0,10,1044,11,1,10,0,363,0,0,17,52404,1,1,1
3,0,0,10,1044,11,1,10,0,363,0,0,17,52404,1,1,1
4,0,0,10,1044,11,1,10,0,363,0,0,17,52404,1,1,1


### - Compare all `not_found`[Reference] values to Extensiv Table

In [15]:

def find_extensiv_reference_columns(ext_table):

    match_dct = dict()
    suffix = 0

    for i, v in enumerate(not_found['Reference']):

        #! Suffix Maker WIP
        if i != 0 and v == not_found['Reference'][i-1]:
            
            suffix += 1
            v = f'{v}-s{suffix}'
            
            
        elif i != 0 and v != not_found['Reference'][i-1]:
            suffix = 0
        else:
            continue 
            
        match_lst = find_col_match(ext_table, not_found['Pattern'][i])

        if match_lst is not None and not pd.isna(v):  
            
            match_dct[v] = {'match_lst': match_lst,
                            'Total Charges': not_found['Total Charges'][i],
                            'Tracking #': not_found['Tracking #'][i]}

    return match_dct


In [16]:
# Test Find column matches

amt_match_dct = find_extensiv_reference_columns(amt)
gp_acoustics_dct = find_extensiv_reference_columns(gp_acoustics)

In [17]:
# Reference numbers in FedEx and matching columns in Extensiv 

gp_acoustics_dct

{10558: {'match_lst': {'AsnCandidate',
   'Column1',
   'CreatedByIdentifier.Id',
   'Custom.TotalResults',
   'CustomerIdentifier.Id',
   'Index',
   'LoadedState',
   'OrderId',
   'ParcelLabelType',
   'ReferenceNum',
   'RouteCandidate',
   'RoutingInfo.BillOfLading',
   'RoutingInfo.Mode',
   'RoutingInfo.TrackingNumber',
   'ShipTo.AddressStatus',
   'ShipTo.ContactId',
   'ShipTo.Zip',
   'Status',
   'TransactionEntryType',
   'WarehouseTransactionSourceType'},
  'Total Charges': 4.92,
  'Tracking #': 280881667358},
 10556: {'match_lst': {'AsnCandidate',
   'Column1',
   'CreatedByIdentifier.Id',
   'Custom.TotalResults',
   'CustomerIdentifier.Id',
   'Index',
   'LoadedState',
   'OrderId',
   'ParcelLabelType',
   'ReferenceNum',
   'RouteCandidate',
   'RoutingInfo.BillOfLading',
   'RoutingInfo.Mode',
   'RoutingInfo.TrackingNumber',
   'ShipTo.AddressStatus',
   'ShipTo.ContactId',
   'ShipTo.Zip',
   'Status',
   'TransactionEntryType',
   'WarehouseTransactionSourceType

### - Compare each `not_found`[Reference] to each value in matched Extensiv columns

In [18]:
# Search through values in each of the matched columns in Extensiv table and look for direct match to Reference

def find_value_match(table, match_dct):

    match_lst = list()

    for reference in match_dct:

        matches = match_dct[reference]['match_lst']
        total_charges = match_dct[reference]['Total Charges']
        tracking_number = match_dct[reference]['Tracking #']

        for col in table[list(matches)]:

            for i, val in enumerate(table[col]):
                    
                base_reference = re.sub(r'-s\d+$', '', str(reference))
                
                if val == reference or val == base_reference:
                    
                    match_entry = {'Reference': reference,
                                   'Name': table['CustomerIdentifier.Name'][i],
                                   'Column': col, 
                                   'Total Charges': total_charges,
                                   'Tracking #': tracking_number}

                    if match_entry not in match_lst:  
                        match_lst.append(match_entry)                        
                else:  
                    continue

    if not match_lst:      
        print(f'No Matches')

    else:
        return match_lst

In [19]:
# amt_found_values = find_value_match(gp_acoustics, gp_acoustics_dct)
gp_found_values = find_value_match(gp_acoustics, gp_acoustics_dct)

In [20]:
gp_found_values

[{'Reference': 10558,
  'Name': 'GP Acoustics',
  'Column': 'OrderId',
  'Total Charges': 4.92,
  'Tracking #': 280881667358},
 {'Reference': 10556,
  'Name': 'GP Acoustics',
  'Column': 'OrderId',
  'Total Charges': 4.92,
  'Tracking #': 280881537650},
 {'Reference': 10562,
  'Name': 'GP Acoustics',
  'Column': 'OrderId',
  'Total Charges': 10.63,
  'Tracking #': 280881473027},
 {'Reference': 11397,
  'Name': 'GP Acoustics',
  'Column': 'OrderId',
  'Total Charges': 9.56,
  'Tracking #': 280768308100},
 {'Reference': 'GP Acoustics',
  'Name': 'GP Acoustics',
  'Column': 'CustomerIdentifier.Name',
  'Total Charges': 11.9,
  'Tracking #': 418132348770},
 {'Reference': 'GP Acoustics-s1',
  'Name': 'GP Acoustics',
  'Column': 'CustomerIdentifier.Name',
  'Total Charges': 11.9,
  'Tracking #': 418132348758},
 {'Reference': 'GP Acoustics-s2',
  'Name': 'GP Acoustics',
  'Column': 'CustomerIdentifier.Name',
  'Total Charges': 11.9,
  'Tracking #': 418132348769},
 {'Reference': 'GP Acoustics-

# **Invoice Data** [`Receiver Info`] <-> **Extensiv** [`Receiver Info`]

In [157]:
# Create list of FedEx References that were found in Extensiv cross-examination

found_reference_lst = list()

for i in gp_found_values:
    found_reference_lst.append(i['Reference'])


,Customer PO #,Reference,Reference 2,Total Charges,Receiver Name,Receiver Company,Receiver Address,Tracking #,Pattern
757,NaN,NaN,NaN,6.54,SPERO PLESSAS,SPERO PLESSAS,10628 GRAND BLVD,779439316825,re.compile('\\w+')
758,NaN,198370,Nautical/Whill/198370,23.35,Salome Macaspac,Whill Inc,200 Sunfish Court,418132350368,re.compile('\\d+')
759,NaN,Nautical/Arrived/Horizon,NaN,7.47,Karen Maxwell,Bartlett Park District Prescho,700 South Bartlett Road,426933217899,re.compile('\\w+/\\w+/\\w+')
760,NaN,Nautical/Arrived/Horizon,NaN,7.47,Karen Maxwell,Bartlett Park District Prescho,700 South Bartlett Road,426933218358,re.compile('\\w+/\\w+/\\w+')
761,NaN,Nautical/Arrived/Horizon,NaN,7.47,Karen Maxwell,Bartlett Park District Prescho,700 South Bartlett Road,426933218369,re.compile('\\w+/\\w+/\\w+')


In [197]:
# Create FedEx null DataFrame 

invoice_data_null = not_found[(not_found['Customer PO #'].isna()) & ~(not_found['Reference'].isin(found_reference_lst))]
invoice_data_null2 = not_found

In [158]:
# Create Extensiv Receiver Information tables

amt_receiver_info = amt[['ShipTo.CompanyName',
               'ShipTo.Name',
               'ShipTo.Address1',
               'CustomerIdentifier.Name']]
gp_receiver_info = gp_acoustics[['ShipTo.CompanyName',
               'ShipTo.Name',
               'ShipTo.Address1',
               'CustomerIdentifier.Name']]

amt_receiver_info_nd = amt_receiver_info.drop_duplicates(['ShipTo.CompanyName',
                                   'ShipTo.Name',
                                   'ShipTo.Address1',
                                   'CustomerIdentifier.Name'])

gp_receiver_info_nd = gp_acoustics.drop_duplicates(['ShipTo.CompanyName',
               'ShipTo.Name',
               'ShipTo.Address1',
               'CustomerIdentifier.Name'])



In [198]:
# Create Dictionaries from Invoice and Extensiv Tables to compare

idn_receiver_entry = {}

for i, row in invoice_data_null2.iterrows():

    idn_receiver_entry[i] = {'Receiver Address': row['Receiver Address'],
                        'Receiver Company': row['Receiver Company'],
                        'Receiver Name': row['Receiver Name'],
                        'Tracking #': row['Tracking #']}

# Create AMT Receiver Info dictionary
  
amt_receiver_entry = dict()

for i, row in amt_receiver_info_nd.iterrows():
    
    amt_receiver_entry[i] = {'Receiver Address': row['ShipTo.Address1'],
                                      'Receiver Company': row['ShipTo.CompanyName'],
                                      'Receiver Name': row['ShipTo.Name'],
                                      'Customer Identifier': row['CustomerIdentifier.Name']}
    

# Create GP Acoustics Receiver Info dictionary

gp_receiver_entry = dict()

for i, row in gp_receiver_info_nd.iterrows():
    
    gp_receiver_entry[i] = {'Receiver Address': row['ShipTo.Address1'],
                                      'Receiver Company': row['ShipTo.CompanyName'],
                                      'Receiver Name': row['ShipTo.Name'],
                                      'Customer Identifier': row['CustomerIdentifier.Name']}


In [202]:
def compare_receiver_info(fedex_receiver_info, ext_receiver_info):

    #! Should the combinations of Receiver Information be exact 'And' or 'Or'
    match_entry = dict()
    match_lst = list()

    for i in fedex_receiver_info:

        for e in ext_receiver_info:

            if fedex_receiver_info[i]['Receiver Address'] == ext_receiver_info[e]['Receiver Address'] or \
                fedex_receiver_info[i]['Receiver Name'] == ext_receiver_info[e]['Receiver Name'] or \
                fedex_receiver_info[i]['Receiver Company'] == ext_receiver_info[e]['Receiver Company']:

                match_entry = {'Address': fedex_receiver_info[i]['Receiver Address'],
                            'Name': fedex_receiver_info[i]['Receiver Name'],
                            'Company': fedex_receiver_info[i]['Receiver Company'],
                            'Customer': ext_receiver_info[e]['Customer Identifier']}
                
                if match_entry not in match_lst:
                    match_lst.append(match_entry)

    if match_lst:
        return match_lst
    else:
        print('No Match')

In [205]:
compare_receiver_info(idn_receiver_entry, gp_receiver_entry)

[{'Address': '13 MONMOUTH ST',
  'Name': 'RELIC MUSIC LLC',
  'Company': 'RELIC MUSIC LLC',
  'Customer': 'GP Acoustics'},
 {'Address': '5479 PERRY DR',
  'Name': 'ISP TECHNOLOGIES LLC',
  'Company': 'ISP TECHNOLOGIES LLC',
  'Customer': 'GP Acoustics'},
 {'Address': '380 LOG CANOE CIR',
  'Name': 'PRS GUITARS',
  'Company': 'PRS GUITARS',
  'Customer': 'GP Acoustics'},
 {'Address': '8 THORNTON RD',
  'Name': 'MUSICAL FULFILMENT SERVICES',
  'Company': 'MUSICAL FULFILMENT SERVICES',
  'Customer': 'GP Acoustics'},
 {'Address': '28 SUNDANCE CIR',
  'Name': 'ATLAS CUSTOM CABINETS',
  'Company': 'ATLAS CUSTOM CABINETS',
  'Customer': 'GP Acoustics'},
 {'Address': '180 CARPENTER AVE',
  'Name': 'ALTO MUSIC',
  'Company': 'ALTO MUSIC',
  'Customer': 'GP Acoustics'},
 {'Address': '71 STEEP RD',
  'Name': 'STEVEN MILLER',
  'Company': 'STEVEN MILLER',
  'Customer': 'GP Acoustics'},
 {'Address': '7091 NW 51ST ST',
  'Name': 'MIAMI PARTS IMPORT,INC',
  'Company': 'MIAMI PARTS IMPORT,INC',
  'Cus

# MISC